# CS416 - Create a Narrative Visualizaton
# COVID19 data preparation

In [1]:
import time
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
from io import StringIO

import numpy as np
import pandas as pd
import requests

In [2]:
# Source: https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary
def xml_to_dict(xml, result):
    for child in xml:
        if len(child) == 0:
            result[child.tag] = child.text
        else:
            if child.tag in result:
                if not isinstance(result[child.tag], list):
                    result[child.tag] = [result[child.tag]]
                result[child.tag].append(xml_to_dict(child, {}))
            else:
                result[child.tag] = xml_to_dict(child, {})
    return result

# 2020-04-13 10:22:29.663 > 2020-04-13
def createDt_to_stateDt(createDt):
    createDt = createDt[0:10]
    createDt = createDt.replace("-", "")
    createDt = createDt.strip()
    return '{}-{}-{}'.format(createDt[0:4], createDt[4:6], createDt[6:8])

# 2021-07-25 > 2021-07-24
def str_yesterday(str_day):
    dt_day = datetime.strptime(str_day, '%Y-%m-%d')
    dt_day = dt_day - timedelta(days=1)
    return dt_day.strftime("%Y-%m-%d")

In [3]:
TODAY = time.strftime("%Y%m%d")

# OWID complete dataset

In [5]:
# COVID-19 (coronavirus) by Our World in Data
# https://github.com/owid/covid-19-data/tree/master/public/data
variants = requests.get('https://covid.ourworldindata.org/data/owid-covid-data.csv')
f = StringIO(variants.text)
pd_csv = pd.read_csv(f)
print('All data country count: {}'.format(len(pd_csv.location.unique())))

korea_idx = pd_csv.location.str.contains('South Korea')
pd_all = pd_csv[korea_idx]
pd_all.set_index('date', inplace=True)

# Our World in Data error fix
pd_all.loc['2021-07-23', 'new_cases'] = 1629
pd_all.loc['2021-07-24', 'new_cases'] = 1486

All data country count: 231


/home/suser/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


# append additional columns (age, gender)

In [6]:
columns = []
columns += ['man_conf_case', 'man_conf_case_rate', 'man_critical_rate', 'man_death', 'man_death_rate']
columns += ['woman_conf_case', 'woman_conf_case_rate', 'woman_critical_rate', 'woman_death', 'woman_death_rate']
for i in range(0, 8):
    start = i*10
    end = start+9
    columns.append('age_{}-{}_conf_case'.format(start, end))
    columns.append('age_{}-{}_conf_case_rate'.format(start, end))
    columns.append('age_{}-{}_critical_rate'.format(start, end))
    columns.append('age_{}-{}_death'.format(start, end))
    columns.append('age_{}-{}_death_rate'.format(start, end))
columns += ['age_over-80_conf_case', 'age_over-80_conf_case_rate', 'age_over-80_critical_rate', 'age_over-80_death', 'age_over-80_death_rate']
for column in columns:
    pd_all.insert(pd_all.columns.size, column, np.nan)

# merge COVID19 variants information

In [7]:
# COVID19 variants information
# https://github.com/owid/covid-19-data/blob/master/public/data/variants/covid-variants.csv
variants = requests.get('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/variants/covid-variants.csv')
f = StringIO(variants.text)
pd_var = pd.read_csv(f)
print('Variants data country count: {}'.format(len(pd_var.location.unique())))

korea_idx = pd_var.location.str.contains('South Korea')
korea = pd_var[korea_idx]
variant_types = korea['variant'].unique()
print('Variants count: {}'.format(len(pd_var.variant.unique())))

Variants data country count: 84
Variants count: 23


In [8]:
# add variants columns
pd_all.insert(pd_all.columns.size, 'var_sequences_total', np.nan)
for variant_nm in variant_types:
    pd_all.insert(pd_all.columns.size, 'var_' + variant_nm + '_num', np.nan)
    pd_all.insert(pd_all.columns.size, 'var_' + variant_nm + '_per', np.nan)

In [9]:
# update variant columns
# Caveats: This data only exists per two weeks
for index, row in korea.iterrows():
    state_dt = row['date']
    variant_nm = row['variant']
    num_sequences_total = row['num_sequences_total']
    num_sequences = row['num_sequences']
    perc_sequences = row['perc_sequences']
    pd_all.loc[state_dt, 'var_sequences_total'] = num_sequences_total
    pd_all.loc[state_dt, 'var_' + variant_nm + '_num'] = num_sequences
    pd_all.loc[state_dt, 'var_' + variant_nm + '_per'] = perc_sequences

# merge aging group information
South Korea's Ministry of Health and Welfare Data Portal provides detailed data by aging group. Anyone can download data after having an authentication key. I will **_not_** upload this authentication key intentionally since there is a API request limitation for each user.

In [10]:
f = open("./OpenAPI_key.txt")
AUTH = f.read()

In [11]:
# COVID19 data by aging group (공공데이터활용지원센터_보건복지부 코로나19 연령별·성별감염 현황)
# https://www.data.go.kr/iim/api/selectAPIAcountView.do
params = {'serviceKey': AUTH, 'pageNo': 1, 'numOfRows': 10, 'startCreateDt': '20200101', 'endCreateDt': '20230101'}
result = requests.get('http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson', params=params)

root = ET.fromstring(result.text)
dictRoot = xml_to_dict(root, {})
resultCode = dictRoot['header']['resultCode']
if resultCode == '00':
    # XML to pandas: https://medium.com/@robertopreste/from-xml-to-pandas-dataframes-9292980b1c1c
    item_list = dictRoot['body']['items']['item']
    new_rows = {}
    for item in item_list:
        state_dt = createDt_to_stateDt(item['createDt'])  # OpenAPI date
        state_dt = str_yesterday(state_dt)  # To standardize with OWID date
        if state_dt not in pd_all.index:
            continue

        gubun = item['gubun']
        if gubun == '남성':
            pd_all.loc[state_dt, 'man_conf_case'] = item['confCase']
            pd_all.loc[state_dt, 'man_conf_case_rate'] = item['confCaseRate']
            pd_all.loc[state_dt, 'man_critical_rate'] = item['criticalRate']
            pd_all.loc[state_dt, 'man_death'] = item['death']
            pd_all.loc[state_dt, 'man_death_rate'] = item['deathRate']
        elif gubun == '여성':
            pd_all.loc[state_dt, 'woman_conf_case'] = item['confCase']
            pd_all.loc[state_dt, 'woman_conf_case_rate'] = item['confCaseRate']
            pd_all.loc[state_dt, 'woman_critical_rate'] = item['criticalRate']
            pd_all.loc[state_dt, 'woman_death'] = item['death']
            pd_all.loc[state_dt, 'woman_death_rate'] = item['deathRate']
        elif gubun == '80 이상':
            pd_all.loc[state_dt, 'age_over-80_conf_case'] = item['confCase']
            pd_all.loc[state_dt, 'age_over-80_conf_case_rate'] = item['confCaseRate']
            pd_all.loc[state_dt, 'age_over-80_critical_rate'] = item['criticalRate']
            pd_all.loc[state_dt, 'age_over-80_death'] = item['death']
            pd_all.loc[state_dt, 'age_over-80_death_rate'] = item['deathRate']
        else:
            pd_all.loc[state_dt, 'age_{}_conf_case'.format(gubun)] = item['confCase']
            pd_all.loc[state_dt, 'age_{}_conf_case_rate'.format(gubun)] = item['confCaseRate']
            pd_all.loc[state_dt, 'age_{}_critical_rate'.format(gubun)] = item['criticalRate']
            pd_all.loc[state_dt, 'age_{}_death'.format(gubun)] = item['death']
            pd_all.loc[state_dt, 'age_{}_death_rate'.format(gubun)] = item['deathRate']

In [12]:
# calculate the difference between days (only applied to people_vaccinated, age fields)
def add_col_diff(pd, diff_col):
    new_col = 'new_' + diff_col
    pd.insert(pd.columns.size, new_col, 0)
    for row in range(1, len(pd_all)):
        before = np.double(np.nan_to_num(pd.loc[pd.index[row - 1], diff_col]))
        after = np.double(np.nan_to_num(pd.loc[pd.index[row], diff_col]))
        diff = after - before
        diff = diff if diff > 0 else 0
        pd.loc[pd.index[row], new_col] = diff

add_col_diff(pd_all, 'people_vaccinated')
add_col_diff(pd_all, 'age_over-80_conf_case')
for i in range(0, 8):
    start = i*10
    end = start+9
    add_col_diff(pd_all, 'age_{}-{}_conf_case'.format(start, end))

In [13]:
# remove last 2 lines (There is a time delays on Our World in Data)
# Remove last columns: https://stackoverflow.com/questions/26921651/how-to-delete-the-last-row-of-data-of-a-pandas-dataframe
pd_all = pd_all.head(-2)

In [14]:
pd_all = pd_all.round(3)
pd_all = pd_all.drop(['iso_code', 'continent', 'location'], axis='columns')
pd_all.to_csv('./covid_kor_all.csv', index=True)

# Post-process
For some reasons, there are many N/A values in the Our World in Data, and OpenAPI data. 
It is hard to fix errors of the raw data; so I modified these data manually.